# Cardiovascular Risk Assessment — Reproducible Notebook

This notebook demonstrates an end-to-end ML workflow using entirely synthetic data. It replicates the methodology from your fracture-risk project (data generation → preprocessing → feature engineering → model training → evaluation → clinical interpretation) while avoiding any original dataset or SINS content.

Guidelines:
- This notebook uses the shared synthetic data generator in `shared/data_generators/cardiovascular_data_generator.py`.
- Do not load or reference any private datasets.
- Run cells sequentially to reproduce results.

In [ ]:
# Section 1: Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Optional high-performance libraries
try:
    import xgboost as xgb
except Exception:
    xgb = None
try:
    import lightgbm as lgb
except Exception:
    lgb = None

# Imbalanced learning
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
except Exception:
    SMOTE = None
    ImbPipeline = None

# Metrics
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, classification_report,
    precision_recall_curve, auc, accuracy_score, recall_score
)

# SHAP (optional)
try:
    import shap
except Exception:
    shap = None

# Display settings
pd.set_option('display.max_columns', 200)
sns.set(style='whitegrid')

print('libraries imported')

## Section 2: Generate Synthetic Dataset

We will generate a synthetic cardiovascular dataset with realistic clinical relationships, using the shared generator. This avoids any use of private or original fracture-risk data.

In [ ]:
# generate synthetic data using the shared generator
from shared.data_generators.cardiovascular_data_generator import generate_cardiovascular_data

# generate a moderate-size dataset for notebook demo
df = generate_cardiovascular_data(n_samples=5000)

# quick peek
print(df.shape)
df.head()

## Section 3: Data Preprocessing and Feature Engineering

Handle missing values, create derived features, and prepare data for modeling.

In [ ]:
# Basic checks
print(df.isna().mean().round(3).sort_values(ascending=False).head(10))

# Derived features examples
# pulse_pressure, total_hdl_ratio, bmi_category

df['pulse_pressure'] = df['systolic_bp'] - df['diastolic_bp']
df['total_hdl_ratio'] = df['total_cholesterol'] / (df['hdl_cholesterol'].replace(0, np.nan))

def bmi_category(bmi):
    if bmi < 18.5:
        return 'underweight'
    if bmi < 25:
        return 'normal'
    if bmi < 30:
        return 'overweight'
    if bmi < 35:
        return 'obese1'
    return 'obese2'

df['bmi_category'] = df['bmi'].apply(bmi_category)

# Convert some binary columns to int (ensure clean types)
binary_cols = ['smoking_status','diabetes','hypertension','family_history_cvd','previous_mi','previous_stroke','on_statin_therapy','on_bp_medication']
for c in binary_cols:
    df[c] = df[c].astype(int)

# Show updated frame
print(df[['pulse_pressure','total_hdl_ratio','bmi','bmi_category']].head())

## Section 4: Exploratory Data Analysis and Visualization

Distribution plots, correlation heatmap, and outcome-stratified boxplots.

In [ ]:
# Outcome distribution
sns.countplot(x='cvd_risk_10yr', data=df)
plt.title('CVD 10yr High-risk (1) vs Low (0)')
plt.show()

# Correlation heatmap for numeric features
numeric_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()

plt.figure(figsize=(14,10))
mask = np.zeros_like(df[numeric_cols].corr())
mask[np.triu_indices_from(mask)] = True
sns.heatmap(df[numeric_cols].corr(), mask=mask, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation heatmap (numeric features)')
plt.show()

# Boxplot of systolic BP by outcome
plt.figure(figsize=(8,4))
sns.boxplot(x='cvd_risk_10yr', y='systolic_bp', data=df)
plt.title('Systolic BP by CVD outcome')
plt.show()

## Section 5: Class Imbalance Analysis

Check class imbalance and demonstrate SMOTE effect (if imblearn available).

In [ ]:
# Class balance
counts = df['cvd_risk_10yr'].value_counts()
print(counts)
print('Imbalance ratio (major/minor):', counts.max() / counts.min())

# If SMOTE is available, show before/after counts on a small sample
if SMOTE is not None:
    X = df.drop(columns=['cvd_risk_10yr','patient_id','assessment_date'])
    y = df['cvd_risk_10yr']
    X_sample, _, y_sample, _ = train_test_split(X, y, stratify=y, test_size=0.9, random_state=42)
    print('Sample counts before:', np.bincount(y_sample))
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X_sample.select_dtypes(include=[np.number]).fillna(0), y_sample)
    print('Sample counts after SMOTE:', np.bincount(y_res))
else:
    print('imblearn not installed — skip SMOTE demo')

## Section 6: Model Pipeline Development

Create preprocessing pipelines (numerical impute/scale, categorical encode) and combine with models. Use stratified splits.

In [ ]:
# Prepare feature lists
exclude = ['patient_id','assessment_date','cvd_risk_10yr']
all_features = [c for c in df.columns if c not in exclude]
num_features = df[all_features].select_dtypes(include=['int64','float64']).columns.tolist()
cat_features = df[all_features].select_dtypes(include=['object','category']).columns.tolist()

print('numerical:', len(num_features), 'categorical:', len(cat_features))

# Preprocessing
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features)
])

# Example model pipelines
models = {
    'logistic': LogisticRegression(max_iter=1000),
    'random_forest': RandomForestClassifier(n_estimators=200, random_state=42)
}

if xgb is not None:
    models['xgboost'] = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
if lgb is not None:
    models['lightgbm'] = lgb.LGBMClassifier()

pipelines = {name: Pipeline([('pre', preprocessor), ('clf', clf)]) for name, clf in models.items()}

# Train/val/test split
X = df[all_features]
y = df['cvd_risk_10yr']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)

print('train/val/test sizes:', X_train.shape[0], X_val.shape[0], X_test.shape[0])

## Section 7: Machine Learning Model Training

Train multiple models with cross-validation; report validation AUC.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}
for name, pipe in pipelines.items():
    print('Training', name)
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    results[name] = scores
    print(f'{name} AUC: mean={scores.mean():.3f} std={scores.std():.3f}')

# Fit final models on full training set
fitted_models = {}
for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    fitted_models[name] = pipe

print('models trained')

## Section 8: Model Performance Evaluation

Evaluate trained models on the test set with multiple metrics and confusion matrices.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

perf = {}
for name, model in fitted_models.items():
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1] if hasattr(model, 'predict_proba') else model.decision_function(X_test)
    auc_score = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    perf[name] = {'auc': auc_score, 'accuracy': acc, 'recall': rec, 'precision': prec, 'f1': f1, 'cm': cm}
    print(f"{name}: AUC={auc_score:.3f}, accuracy={acc:.3f}, recall={rec:.3f}, precision={prec:.3f}, f1={f1:.3f}")

# Show comparison table
pd.DataFrame({k: {m: v for m, v in perf[k].items() if m!='cm'} for k in perf}).T

## Section 9: Feature Importance Analysis

Extract feature importances from tree-based models and compare ranks.

In [ ]:
# Get feature names from preprocessor
num_out = num_features
cat_out = list(pipelines['random_forest'].named_steps['pre'].named_transformers_['cat'].named_steps['ohe'].get_feature_names_out(cat_features)) if len(cat_features)>0 else []
feature_names = num_out + cat_out

# Random Forest importances
if 'random_forest' in fitted_models:
    rf = fitted_models['random_forest'].named_steps['clf']
    importances = rf.feature_importances_
    fi = pd.Series(importances, index=feature_names).sort_values(ascending=False).head(20)
    plt.figure(figsize=(8,6))
    fi.plot.barh()
    plt.gca().invert_yaxis()
    plt.title('Random Forest - Top 20 feature importances')
    plt.show()
else:
    print('Random forest not trained')

# SHAP example (if available)
if shap is not None and 'xgboost' in fitted_models:
    explainer = shap.TreeExplainer(fitted_models['xgboost'].named_steps['clf'])
    X_num = fitted_models['xgboost'].named_steps['pre'].transform(X_test)[:, :len(num_out)]
    shap_values = explainer.shap_values(X_num)
    shap.summary_plot(shap_values, features=X_test[num_out], feature_names=num_out)
else:
    print('SHAP or XGBoost not available for summary plot')

## Section 10: Clinical Threshold Analysis

Implement a simple clinical rule (composite score) and compare operating points vs model probabilities.

In [ ]:
# Simple clinical score: age_points + bp + smoking + diabetes

def clinical_score(row):
    score = 0
    score += max(0, (row['age'] - 45) / 5)
    score += 1 if row['systolic_bp'] > 140 else 0
    score += 1 if row['smoking_status'] == 1 else 0
    score += 2 if row['diabetes'] == 1 else 0
    return score

X_test2 = X_test.copy()
X_test2['clinical_score'] = X_test2.apply(clinical_score, axis=1)

# Evaluate rule at threshold >=2
rule_pred = (X_test2['clinical_score'] >= 2).astype(int)
print('Clinical rule performance:')
print('accuracy', accuracy_score(y_test, rule_pred))
print('recall', recall_score(y_test, rule_pred))
print('precision', precision_score(y_test, rule_pred))

## Section 11: Results Comparison and Visualization

Compare ROC curves, precision-recall, and a summary table of all models and the clinical rule.

In [ ]:
# ROC curves
plt.figure(figsize=(8,6))
for name, model in fitted_models.items():
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test)[:,1]
    else:
        y_proba = model.decision_function(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=f"{name} (AUC={perf[name]['auc']:.2f})")

# clinical rule as a point
from sklearn.metrics import roc_curve
rule_scores = X_test2['clinical_score']
fpr_r, tpr_r, _ = roc_curve(y_test, rule_scores)
plt.plot(fpr_r, tpr_r, label='Clinical rule', linestyle='--')

plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Comparison')
plt.legend()
plt.show()

# Summary table including clinical rule
summary = pd.DataFrame({name: {'auc': perf[name]['auc'], 'accuracy': perf[name]['accuracy'], 'recall': perf[name]['recall'], 'precision': perf[name]['precision'], 'f1': perf[name]['f1']} for name in perf}).T
summary.loc['clinical_rule'] = [roc_auc_score(y_test, X_test2['clinical_score']), accuracy_score(y_test, rule_pred), recall_score(y_test, rule_pred), precision_score(y_test, rule_pred), f1_score(y_test, rule_pred)]
summary

### Next steps and notes
- Tidy hyperparameter tuning with Optuna/RandomizedSearchCV.
- Add calibration plots and decision curve analysis for clinical usefulness.
- Add fairness checks across demographics (gender, ethnicity).
- Save best model artifacts to `models/trained_models` and create a small FastAPI wrapper in `src/api` for demo.

This notebook is intentionally synthetic and parallel to your fracture-risk notebook. Use it as a reproducible portfolio artifact.

# Visualizations: ROC, Calibration, Feature Importance

This section demonstrates model evaluation and explainability visuals using the saved model artifact: ROC curve, calibration (reliability) plot, and feature importances. It also shows a short scoring demo using a few synthetic patient records.

In [ ]:
# Setup: imports, load saved model artifact, and prepare data for evaluation
import sys
from pathlib import Path
import json
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc, RocCurveDisplay, calibration_curve, brier_score_loss
%matplotlib inline

# Resolve repo root and ensure it is on sys.path
repo_root = Path.cwd()
if not (repo_root / "shared").exists():
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root))

from shared.data_generators.cardiovascular_data_generator import generate_cardiovascular_data
import etl
import cleaning

models_dir = repo_root / "projects" / "02-cardiovascular-risk-ml" / "models"
meta_path = models_dir / "metadata.json"
if not meta_path.exists():
    raise FileNotFoundError("metadata.json not found. Run the training demo (save_and_demo.py) first.")

meta = json.loads(meta_path.read_text())
model_file = models_dir / f"cardio_best_{meta['best_model']}.joblib"
obj = joblib.load(model_file)
model = obj['model']
features = obj['features']
target = obj.get('target', 'cvd_risk_10yr')

# Generate a reproducible synthetic evaluation dataset and run ETL/cleaning
print('Generating synthetic evaluation data (n=1000)')
df = generate_cardiovascular_data(n_samples=1000)
df = etl.basic_etl(df)
df = cleaning.impute_missing(df)

# Prepare X and y
X = df[features]
y = df[target]
print('Data prepared. X shape:', X.shape, 'y distribution:', y.value_counts().to_dict())

In [ ]:
# ROC curve and AUC
probs = model.predict_proba(X)[:, 1] if hasattr(model, 'predict_proba') else model.predict(X)
fpr, tpr, _ = roc_curve(y, probs)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.3f})', linewidth=2)
plt.plot([0,1], [0,1], '--', color='grey', alpha=0.7)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

print(f'AUC = {roc_auc:.3f}')

In [ ]:
# Calibration plot and Brier score
prob_true, prob_pred = calibration_curve(y, probs, n_bins=10)
plt.figure(figsize=(6,5))
plt.plot(prob_pred, prob_true, marker='o', linewidth=2, label='Calibration')
plt.plot([0,1], [0,1], '--', color='grey', alpha=0.7)
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration plot')
plt.legend()
plt.grid(True)
plt.show()

brier = brier_score_loss(y, probs)
print(f'Brier score: {brier:.4f}')

In [ ]:
# Feature importance (for tree-based models) and simple scoring demo
importances = None
if hasattr(model, 'feature_importances_'):
    importances = getattr(model, 'feature_importances_')
elif hasattr(getattr(model, 'estimators_', None), '__len__'):
    # RandomForest may expose ensemble; average importances
    try:
        importances = np.mean([est.feature_importances_ for est in model.estimators_], axis=0)
    except Exception:
        importances = None

if importances is not None:
    fi = pd.Series(importances, index=features).sort_values(ascending=False).head(20)
    plt.figure(figsize=(8,6))
    sns.barplot(x=fi.values, y=fi.index)
    plt.title('Top feature importances')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
else:
    print('Feature importances not available for this model type.')

# Scoring demo: show predictions for first 5 records
print('\nScoring demo (first 5 records):')
X_demo = X.iloc[:5]
if hasattr(model, 'predict_proba'):
    p_demo = model.predict_proba(X_demo)[:, 1]
    for i, p in enumerate(p_demo):
        print(f'sample {i}: P(high CVD risk) = {p:.3f}')
else:
    preds = model.predict(X_demo)
    for i, p in enumerate(preds):
        print(f'sample {i}: pred = {p}')


### Bonus: Logistic coefficients, feature importance table, and interactive ROC

This final cell shows how to extract coefficient-based importance for linear models (logistic regression), a unified importance table (works for tree-based or linear models), and an interactive ROC curve using Plotly if available. If Plotly is not installed the code will print a helpful message and fall back to the static ROC shown earlier.

In [ ]:
# Attempt to use helper functions added to `ml_analysis.py` to show feature importance and an interactive ROC
try:
    from ml_analysis import get_feature_importance, plot_interactive_roc
except Exception:
    # ensure src path is available
    import sys
    from pathlib import Path
    src_path = Path.cwd() / "projects" / "02-cardiovascular-risk-ml" / "src"
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    from ml_analysis import get_feature_importance, plot_interactive_roc

# unwrap pipeline to a fitted estimator if necessary
def _unwrap_model(m):
    try:
        if hasattr(m, 'named_steps'):
            return m.named_steps.get('clf', m)
    except Exception:
        pass
    return m

est = _unwrap_model(model)

# Feature importance / coefficients
fi_df = get_feature_importance(est, features)
print('Top features by importance (head):')
display(fi_df.head(30))

# If logistic (coef_ present) show signed coefficients
if hasattr(est, 'coef_'):
    import pandas as pd
    coef = est.coef_
    # flatten for binary
    coef_vec = coef.ravel() if coef.ndim > 1 else coef
    coef_df = pd.DataFrame({'feature': features, 'coef': coef_vec})
    coef_df['abs_coef'] = coef_df['coef'].abs()
    coef_df = coef_df.sort_values('abs_coef', ascending=False).reset_index(drop=True)
    print('\nLogistic / linear model coefficients (top):')
    display(coef_df.head(20))

# Interactive ROC (Plotly) with graceful fallback
try:
    fig = plot_interactive_roc(est, X, y)
    try:
        # in notebook this will render interactively
        fig.show()
    except Exception:
        # still write to file for manual inspection
        out = str(Path.cwd() / 'projects' / '02-cardiovascular-risk-ml' / 'models' / 'roc_interactive.html')
        fig.write_html(out)
        print(f'Interactive ROC saved to: {out}')
except Exception as e:
    print('Interactive ROC unavailable:', e)
    print('To enable interactive plots install plotly: pip install plotly')
